In [2]:
from debug_utils import detect_ball, load_data
from padel_utils import load_fisheye_params, triangulate_points
import numpy as np

In [ ]:
# K1,D1 = load_fisheye_params("parameters/pallina1-fisheye.txt")
# H1 = np.loadtxt("parameters/pallina1-perspective.txt")

# K2,D2 = load_fisheye_params("parameters/pallina2-fisheye.txt")
# H2 = np.loadtxt("parameters/pallina2-perspective.txt")

# path1 = detect_ball("input_videos/primo test pallina/video1.mp4", K,D,H, "ball1.csv", skip_frame=3, sync_frame=1)
# path2 = detect_ball("input_videos/primo test pallina/video2.mp4", K,D,H, "ball2.csv"


# Suppose we have already detected the ball in the two videos and saved the results in the csv files
path1 = "output_data/ball1-0area.csv"
path2 = "output_data/ball2-0area.csv"

In [4]:
O1 = np.array([5, 20.325, 2.635])
O2 = np.array([5, -0.325, 2.635])

In [5]:
data1 = load_data(path1)
data2 = load_data(path2)

In [ ]:
import csv

out_path = 'output_data/triangulated_points.csv'

with open(out_path, "w"):
    pass

writer = csv.writer(open(out_path, "w"))

for frame in range(len(data1)):
    row = [frame]
    points, errors = triangulate_points(O1, O2, data1[frame], data2[frame])
    if points is not None:
        for p, e in zip(points, errors):
            row.append(p)
            row.append(e)
    
    writer.writerow(row)
    

# Show the video with balls drawn on top

In [ ]:
# Read the csv file
import csv
out_path = 'output_data/triangulated_points.csv'

data = []
with open(out_path, "r") as f:
    reader = csv.reader(f)
    for row in reader:
        frame_num = int(row[0])

        detections = []
        for i in range(1, len(row), 2):
            detection_pos_str = row[i].strip('[] ')
            detection_pos = [float(coord) for coord in detection_pos_str.split()]
            detection_error = float(row[i+1])
            
            detections.append((detection_pos, detection_error))
        # frame_data = (frame_num, detections)
        # data.append(frame_data)
        data.append(detections)

In [3]:
import cv2
from padel_utils import draw_mini_court
import numpy as np

video_path = "input_videos/primo test pallina/video1.mp4"
skip_frame = 3
sync_frame = 1

cap = cv2.VideoCapture(video_path)

for _ in range(skip_frame):
    cap.read()

for frame in range(len(data)):
    cap.read()
    ret, img = cap.read()
    if not ret:
        break
    
    detections = data[frame]
    positions = [d[0] for d in detections]
    errors = [d[1] for d in detections]

    best_position = None
    if len(positions) != 0:
        best_detection = np.argmin(errors)
        best_position = positions[best_detection]

    img = draw_mini_court(img, None, detections)

    cv2.imshow("Frame", img)
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()